In [2]:
import os

import cvxpy as cvx
import numpy as np
import pandas as pd

from cvxportfolio.policies import SinglePeriodOpt
from cvxportfolio.costs import HcostModel, TcostModel
from cvxportfolio.returns import ReturnsForecast
from cvxportfolio.risks import FullSigma
from cvxportfolio.tests.base_test import BaseTest

In [1]:
DIR = '/home/ldf/pkg_source/cvxgrp/cvxportfolio/cvxportfolio/tests/'

In [3]:
sigma = pd.read_csv(DIR + 'sigmas.csv', index_col=0, parse_dates=[0])

In [18]:
sigma.tail()

,AAPL,AXP,BA,CAT,CSCO,CVX,DD,DIS,GS,HD,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-24,0.006211,0.008756,0.016323,0.007555,0.010493,0.018136,0.008092,0.007885,0.008844,0.007856,...,0.011513,0.012943,0.007899,0.007988,0.007632,0.007549,0.008988,0.012907,0.010236,0.007810
2014-12-26,0.021918,0.006329,0.012886,0.009930,0.007719,0.013380,0.008877,0.008289,0.005547,0.008052,...,0.012188,0.009876,0.016960,0.007995,0.005592,0.008155,0.005942,0.010025,0.004588,0.012968
2014-12-29,0.009323,0.009379,0.011753,0.007319,0.008750,0.013988,0.009983,0.012926,0.010865,0.014695,...,0.010883,0.010650,0.007590,0.009954,0.010038,0.011455,0.008437,0.009899,0.010253,0.007695
2014-12-30,0.015888,0.004552,0.005745,0.011962,0.005969,0.013198,0.007469,0.010471,0.009771,0.008497,...,0.016380,0.006989,0.007939,0.007757,0.006907,0.007859,0.008213,0.008667,0.009246,0.007460
2014-12-31,0.025811,0.018470,0.018443,0.022657,0.021119,0.015709,0.019124,0.015599,0.015546,0.013771,...,0.020868,0.016084,0.017666,0.016424,0.019455,0.025559,0.016595,0.018971,0.014768,0.018070


# 如何计算sigma

In [5]:
returns = pd.read_csv(DIR + 'returns.csv', index_col=0, parse_dates=[0])

In [6]:
returns.tail()

,AAPL,AXP,BA,CAT,CSCO,CVX,DD,DIS,GS,HD,...,NKE,PFE,PG,TRV,UNH,UTX,V,VZ,WMT,cash
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-24,-0.004709,0.002771,0.009306,0.002353,0.001770,-0.004212,-0.005320,-0.002851,0.001995,-0.001928,...,0.003334,-0.000954,-0.001608,-0.002150,-0.001653,-0.002559,0.008935,0.000000,-0.002654,0.000005
2014-12-26,0.017677,0.002232,0.002972,0.005442,0.001767,-0.001939,0.004546,0.006461,-0.002246,0.001835,...,0.005608,0.007320,0.003436,-0.000187,-0.001266,0.002823,-0.003774,0.003986,0.005554,0.000005
2014-12-29,-0.000702,-0.000212,0.005014,-0.005518,0.003880,0.000618,-0.003993,0.004946,0.003786,0.007518,...,0.004854,-0.008215,-0.008774,0.002343,-0.002145,-0.003241,-0.004463,-0.006477,-0.003107,0.000005
2014-12-30,-0.012203,0.000000,-0.003477,-0.011952,-0.003514,-0.001853,-0.003207,-0.008377,-0.002447,-0.002487,...,-0.004317,-0.000319,-0.002591,-0.000187,0.005082,-0.005648,-0.003315,-0.004627,0.001731,0.000005
2014-12-31,-0.019019,-0.013048,-0.014033,-0.011448,-0.019041,-0.008222,-0.008714,-0.005385,-0.009606,0.006713,...,-0.007535,-0.007330,-0.014178,-0.010100,-0.017112,-0.010327,-0.008883,-0.011621,-0.010485,0.000005


In [7]:
volume = pd.read_csv(DIR + 'volumes.csv', index_col=0, parse_dates=[0])

In [8]:
volume.tail()

,AAPL,AXP,BA,CAT,CSCO,CVX,DD,DIS,GS,HD,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-24,1.576484e+09,1.208933e+08,2.662852e+08,1.447736e+08,3.053127e+08,4.811707e+08,8.791111e+07,2.005512e+08,1.330897e+08,1.827034e+08,...,5.285054e+08,1.155824e+08,3.229033e+08,2.114478e+08,5.571465e+07,9.522151e+07,1.145835e+08,3.466391e+08,3.448969e+08,2.064882e+08
2014-12-26,3.736315e+09,1.320187e+08,2.829731e+08,2.745445e+08,2.610486e+08,4.637129e+08,1.301787e+08,2.927744e+08,1.944040e+08,2.610639e+08,...,6.065360e+08,1.498095e+08,2.586508e+08,2.780162e+08,8.033448e+07,2.020558e+08,1.802454e+08,3.144696e+08,3.108477e+08,2.707554e+08
2014-12-29,3.055835e+09,1.359416e+08,3.446837e+08,2.091913e+08,3.617781e+08,6.401126e+08,1.435272e+08,3.357244e+08,2.342334e+08,3.540984e+08,...,6.576416e+08,1.923503e+08,3.778113e+08,4.076460e+08,8.517301e+07,1.704991e+08,2.802544e+08,3.454702e+08,3.895651e+08,2.995115e+08
2014-12-30,3.268198e+09,1.306341e+08,3.092807e+08,2.764586e+08,4.152602e+08,5.983789e+08,1.194224e+08,3.241536e+08,2.176284e+08,2.960892e+08,...,7.394724e+08,1.976692e+08,3.287555e+08,4.129013e+08,9.704932e+07,1.773348e+08,2.436852e+08,3.128656e+08,4.363030e+08,2.717213e+08
2014-12-31,4.442247e+09,2.086432e+08,3.379175e+08,2.778835e+08,5.649269e+08,6.723457e+08,1.359183e+08,4.431365e+08,2.972229e+08,5.926924e+08,...,9.609134e+08,2.222279e+08,4.416075e+08,4.915635e+08,1.089152e+08,2.281700e+08,2.617090e+08,3.829259e+08,6.056159e+08,3.418253e+08


In [9]:
a, b, s = 0.0005, 1., 0.

In [10]:
s = s + 1e-3

In [11]:
universe = returns.columns

In [12]:
times = returns.index

In [14]:
# Alpha source
gamma = 100.
n = len(universe)
alpha_model = ReturnsForecast(returns)
emp_Sigma = np.cov(returns.as_matrix().T) + np.eye(n) * 1e-3
risk_model = FullSigma(emp_Sigma)
tcost_model = TcostModel(0, b, sigma, volume, power=2)
hcost_model = HcostModel(s * 0, s)
pol = SinglePeriodOpt(
    alpha_model, [gamma * risk_model, tcost_model, hcost_model], [],
    solver=cvx.ECOS)

In [28]:
# 表达式为函数
type(alpha_model.weight_expr)

method

In [17]:
t = times[1]
p_0 = pd.Series(index=universe, data=1E6) # 期初投资组合
z = pol.get_trades(p_0, t)
z

AAPL    1.114827e+06
AXP    -9.450007e+04
BA      9.838637e+05
CAT    -1.468050e+06
CSCO    6.399648e+05
CVX     9.050438e+04
DD     -1.346487e+06
DIS    -3.905080e+05
GS      1.092952e+06
HD     -8.310433e+05
IBM     3.283627e+04
INTC   -1.359671e+06
JNJ     1.086393e+06
JPM     9.260984e+05
KO      1.207013e+05
MCD    -2.151428e+05
MMM    -5.424439e+05
MRK     3.973008e+05
MSFT   -2.470928e+06
NKE    -7.796548e+05
PFE     5.122819e+05
PG      1.187658e+06
TRV     2.858055e+05
UNH    -1.197355e+06
UTX     2.404410e+05
V      -7.523026e+05
VZ      1.452169e+06
WMT    -7.436850e+04
cash    1.358658e+06
dtype: float64

In [25]:
# Compare with CP calculation.
h = z + p_0
rho = b*sigma.loc[t]*(sum(p_0)/volume.loc[t])
rho = np.hstack([rho, 0])
A = 2*gamma*emp_Sigma + 2*np.diag(rho)
s_val = pd.Series(index=returns.columns, data=s)
s_val['cash'] = 0.
b = returns.loc[t] + 2*rho*(p_0/sum(p_0)) + s_val
h0 = np.linalg.solve(A, b)
offset = np.linalg.solve(A, np.ones(n))
nu = (1 - h0.sum())/offset.sum()
hstar = h0 + nu*offset
hstar.sum()

1.0

In [26]:
hstar

array([ 0.07292508,  0.03122414,  0.06840909, -0.01613966,  0.05655051,
        0.03760361, -0.01194783,  0.02101697,  0.07217077,  0.0058261 ,
        0.03561498, -0.01240243,  0.07194458,  0.06641718,  0.03864487,
        0.02706404,  0.0157778 ,  0.04818279, -0.05072165,  0.00759812,
        0.05214765,  0.07543648,  0.04433812, -0.00680534,  0.04277383,
        0.00854129,  0.08455755,  0.03191832,  0.08133305])

In [27]:
h/sum(p_0)

AAPL    0.072925
AXP     0.031224
BA      0.068409
CAT    -0.016140
CSCO    0.056551
CVX     0.037604
DD     -0.011948
DIS     0.021017
GS      0.072171
HD      0.005826
IBM     0.035615
INTC   -0.012402
JNJ     0.071945
JPM     0.066417
KO      0.038645
MCD     0.027064
MMM     0.015778
MRK     0.048183
MSFT   -0.050722
NKE     0.007598
PFE     0.052148
PG      0.075436
TRV     0.044338
UNH    -0.006805
UTX     0.042774
V       0.008541
VZ      0.084558
WMT     0.031918
cash    0.081333
dtype: float64

In [29]:
# 理想化，不考虑交易成本、持有成本
tcost_model = TcostModel(0)
hcost_model = HcostModel(0)
pol = SinglePeriodOpt(
    alpha_model, [gamma * risk_model, tcost_model, hcost_model], [],
    solver=cvx.ECOS)

In [30]:
t = times[2]
p_0 = pd.Series(index=universe, data=1E6) # 期初投资组合
z = pol.get_trades(p_0, t)
z

AAPL   -1.550660e+06
AXP    -1.677341e+06
BA      1.189445e+06
CAT    -4.649578e+05
CSCO    1.098864e+06
CVX     3.665051e+05
DD     -2.109699e+06
DIS    -4.177143e+04
GS     -1.744456e+06
HD      6.112700e+04
IBM     1.943257e+06
INTC   -2.930007e+05
JNJ     2.103456e+06
JPM    -2.521099e+06
KO     -1.747604e+04
MCD     3.191641e+05
MMM    -9.061603e+05
MRK     2.659996e+05
MSFT    1.676454e+05
NKE    -7.550228e+05
PFE     1.074799e+05
PG      9.984433e+05
TRV    -1.285217e+06
UNH     3.222326e+06
UTX     7.569820e+04
V      -2.850865e+04
VZ      1.243280e+06
WMT    -5.563489e+04
cash    2.883134e+05
dtype: float64